#### Ввод числовых id двух людей

In [1]:
profile_names = [476459668, 265329539]

#### Импорт библиотек и модулей парсинга

In [2]:
import os
import json
from lab1.parser_vk.lib.vk_friends_parser import VKClient, VKFriendsParser
from dotenv import load_dotenv
import random
import pandas as pd
import requests
import time
from datetime import datetime
from collections import defaultdict
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import pickle
from collections import defaultdict

#### Скачивание информации

В этой ячейке в файл json скачивается информация о двух пользователях, их друзьях первого уровня (парсер для второго уровня в частности реализован в main.py).

В people расположены признаки людей, а в edges id дружеских связей.

Парсер импортирован из lab1.parser_vk.lib.vk_friends_parser

Также тут реализовано создание файлов nodes_inference.json и edges_inference.csv

In [4]:
# edges = pd.read_csv('edges.csv')
# sampled_edges = edges.drop_duplicates(subset='target_user_id')
# sampled_edges = sampled_edges.sample(n=100)
# set_ids = list(set(sampled_edges['target_user_id']) | set(sampled_edges['interactor_id']))


# def nodes_maker(profile_names):


def init_inference_files():
    nodes_path = "nodes_inference.json"
    with open(nodes_path, "w", encoding="utf-8") as f:
        json.dump([], f, ensure_ascii=False, indent=2)

    edges_path = "edges_inference.csv"
    with open(edges_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["target_user_id", "interactor_id", "likes_count", "comments_count", "reposts_count"])
    posts_path = "posts_count.csv"
    with open(posts_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["id", "posts"])

    print("Файлы инициализированы:")
    print(f" - {nodes_path}")
    print(f" - {edges_path}")
    print(f" - {posts_path}")

if __name__ == "__main__":
    init_inference_files()

load_dotenv()

VK_SERVICE_ACCESS_TOKEN = "6fdea0906fdea0906fdea0900e6ce5f2d866fde6fdea0900738eb840c756af0caf0aace"
# VK_SERVICE_ACCESS_TOKEN = os.getenv("VK_SERVICE_ACCESS_TOKEN", None)
# if VK_SERVICE_ACCESS_TOKEN is None:
#     raise ValueError("VK_SERVICE_ACCESS_TOKEN не найден в переменных окружения")

vk = VKClient(token=VK_SERVICE_ACCESS_TOKEN)


if os.path.exists("nodes_inference.json"):
    with open("nodes_inference.json", 'r', encoding='utf-8') as f:
        try:
            all_data = json.load(f)
        except json.JSONDecodeError:
            all_data = []
else:
    all_data = []

for profile_name in profile_names:
    root_id = profile_name

    parser = VKFriendsParser(
        vk_client=vk, save_photos=False
    )
    parser.fetch_network_fast([root_id], depth=1)

    temp_filename = f"user_{root_id}.json"
    parser.save_json(temp_filename)
    print(f"Данные сохранены в {temp_filename}")
    
    with open(temp_filename, 'r', encoding='utf-8') as f:
        try:
            user_data = json.load(f)
        except json.JSONDecodeError as e:
            print(f"Ошибка чтения {temp_filename}: {e}")
            continue
    print('++++++++++++++++')


    if (user_data['people']):
        # random_friend = random.choice(user_data['people'])
        # random_friend_id = random.choice(user_data['people'])['id']
        # print(random_friend)
        # print(random_friend_id)
        # parser.fetch_network_fast([random_friend_id], depth=1)

        # # Сохраняем во временный файл
        # temp_filename2 = f"user_{random_friend_id}.json"
        # parser.save_json(temp_filename2)
        # print(f"Данные сохранены в {temp_filename2}")
        
        # Читаем данные из временного файла
        # with open(temp_filename2, 'r', encoding='utf-8') as f:
        #     try:
        #         user_data2 = json.load(f)
        #     except json.JSONDecodeError as e:
        #         print(f"Ошибка чтения {temp_filename2}: {e}")
        #         continue


        if not isinstance(all_data, dict):
            all_data = {"people": [], "edges": []}

        if isinstance(user_data, dict) and "people" in user_data and "edges" in user_data:
            all_data["people"].extend(user_data["people"])
            all_data["edges"].extend(user_data["edges"])
            # all_data["people"].extend(user_data2["people"])
            # all_data["edges"].extend(user_data2["edges"])
        else:
            print(f"Предупреждение: Некорректная структура данных для {profile_name}")
        
        if os.path.exists(temp_filename):
            os.remove(temp_filename)
        # if os.path.exists(temp_filename2):
        #     os.remove(temp_filename2)
        print(f"Данные пользователя {profile_name} добавлены в общий список")

with open("nodes_inference.json", 'w', encoding='utf-8') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=5)

print("Все данные объединены в nodes_inference.json")


# with open('nodes.json', 'r', encoding='utf-8') as f:
#     json_file = json.load(f)

# nodes_ids = pd.json_normalize(json_file['people'])['id'].unique()

# for i in range(len(nodes_ids)):
#     print(i)
#     print(i/len(nodes_ids))
#     node_id = nodes_ids[i]
#     edges_parser(node_id)

Файлы инициализированы:
 - nodes_inference.json
 - edges_inference.csv
 - posts_count.csv
=== Глубина 1/1, обрабатываем 1 пользователей ===
Данные сохранены в user_476459668.json
++++++++++++++++
Данные пользователя 476459668 добавлены в общий список
=== Глубина 1/1, обрабатываем 1 пользователей ===
Данные сохранены в user_265329539.json
++++++++++++++++
Данные пользователя 265329539 добавлены в общий список
Все данные объединены в nodes_inference.json


#### Скачивание в csv количества комментариев, лайков и репостов

In [5]:
class VKParser:
    def __init__(self, access_token, version='5.131'):
        self.access_token = access_token
        self.version = version
        self.base_url = 'https://api.vk.com/method/'
        
    def make_request(self, method, params):
        """Базовый метод для запросов к VK API"""
        url = f"{self.base_url}{method}"
        params.update({
            'access_token': self.access_token,
            'v': self.version
        })
        
        try:
            response = requests.get(url, params=params)
            data = response.json()
            
            if 'error' in data:
                print(f"Ошибка VK API: {data['error']['error_msg']}")
                return None
                
            return data['response']
        except Exception as e:
            print(f"Ошибка запроса: {e}")
            return None

    def resolve_screen_name(self, screen_name):
        print(f"Преобразуем короткое имя '{screen_name}' в ID...")
        
        response = self.make_request('utils.resolveScreenName', {
            'screen_name': screen_name
        })
        
        if response and 'object_id' in response:
            user_id = response['object_id']
            print(f"Найден ID: {user_id}")
            return user_id
        else:
            print(f"Не удалось найти ID для '{screen_name}'")
            return None

    def get_user_id(self, user_input):

        if isinstance(user_input, int) or (isinstance(user_input, str) and user_input.isdigit()):
            return int(user_input)
        
        # Если это короткое имя, преобразуем в ID
        if isinstance(user_input, str) and not user_input.startswith('-'):
            return self.resolve_screen_name(user_input)
        
        return user_input

    def get_wall_posts(self, owner_id, count=100):
        """Получает посты со стены пользователя"""
        print(f"Получаем посты со стены пользователя {owner_id}...")
        
        # Преобразуем owner_id в числовой формат если нужно
        numeric_owner_id = self.get_user_id(owner_id)
        if numeric_owner_id is None:
            print(f"Не удалось определить ID для {owner_id}")
            return []
        
        posts = []
        offset = 0
        max_posts = count
        
        while len(posts) < max_posts:
            response = self.make_request('wall.get', {
                'owner_id': numeric_owner_id,
                'count': min(100, max_posts - len(posts)),
                'offset': offset,
                'extended': 1
            })
            
            if not response or 'items' not in response:
                break
                
            posts.extend(response['items'])
            offset += len(response['items'])
            
            if len(response['items']) == 0:
                break
                
            time.sleep(0.34)
        
        print(f"Получено {len(posts)} постов")

        posts_count = pd.read_csv('posts_count.csv')

        new_row = pd.DataFrame({
            'id': [numeric_owner_id],
            'posts': [len(posts)]
        })

        posts_count = pd.concat([posts_count, new_row], ignore_index=True)
        posts_count.to_csv('posts_count.csv', index=False)



        return posts, numeric_owner_id
    
    def get_likes(self, owner_id, item_id, item_type='post'):
        """Получает список пользователей, лайкнувших запись"""
        likes = []
        offset = 0
        count = 1000
        
        while True:
            response = self.make_request('likes.getList', {
                'type': item_type,
                'owner_id': owner_id,
                'item_id': item_id,
                'count': count,
                'offset': offset,
                'filter': 'likes'
            })
            
            if not response or 'items' not in response:
                break
                
            likes.extend(response['items'])
            offset += len(response['items'])
            
            if len(response['items']) < count:
                break
                
            time.sleep(0.34)
        
        return likes
    
    def get_comments(self, owner_id, post_id):
        """Получает комментарии к посту"""
        comments = []
        offset = 0
        count = 100
        
        while True:
            response = self.make_request('wall.getComments', {
                'owner_id': owner_id,
                'post_id': post_id,
                'count': count,
                'offset': offset,
                'extended': 0
            })
            
            if not response or 'items' not in response:
                break
                
            for comment in response['items']:
                comment_data = {
                    'id': comment['id'],
                    'from_id': comment['from_id'],
                    'date': comment['date'],
                    'text': comment['text'],
                    'likes': comment.get('likes', {}).get('count', 0)
                }
                comments.append(comment_data)
            
            offset += len(response['items'])
            
            if len(response['items']) < count:
                break
                
            time.sleep(0.34)
        
        return comments
    
    def get_reposts(self, owner_id, post_id):
        """Получает информацию о репостах"""
        response = self.make_request('wall.getReposts', {
            'owner_id': owner_id,
            'post_id': post_id,
            'count': 1000
        })
        
        if not response:
            return []
            
        reposts = []
        if 'items' in response:
            for repost in response['items']:
                repost_data = {
                    'id': repost['id'],
                    'from_id': repost['from_id'],
                    'date': repost['date'],
                    'text': repost.get('text', ''),
                    'copy_history': repost.get('copy_history', [])
                }
                reposts.append(repost_data)
        
        return reposts
    
    def get_user_info(self, user_ids):
        """Получает информацию о пользователях"""
        if not user_ids:
            return {}
            
        numeric_ids = []
        for user_id in user_ids:
            if isinstance(user_id, int) or (isinstance(user_id, str) and user_id.lstrip('-').isdigit()):
                numeric_ids.append(str(user_id))
        
        if not numeric_ids:
            return {}
            
        response = self.make_request('users.get', {
            'user_ids': ','.join(numeric_ids),
            'fields': 'first_name,last_name,sex,bdate,city,country,screen_name'
        })
        
        if not response:
            return {}
            
        user_info = {}
        for user in response:
            user_info[user['id']] = {
                'first_name': user.get('first_name', ''),
                'last_name': user.get('last_name', ''),
                'sex': user.get('sex', 0),
                'bdate': user.get('bdate', ''),
                'city': user.get('city', {}).get('title', '') if 'city' in user else '',
                'country': user.get('country', {}).get('title', '') if 'country' in user else '',
                'screen_name': user.get('screen_name', '')
            }
        
        return user_info
    
    def process_post(self, post, owner_id):
        """Обрабатывает один пост и собирает всю информацию"""
        post_id = post['id']
        
        print(f"Обрабатываем пост {post_id}...")
        
        post_data = {
            'post_id': post_id,
            'owner_id': owner_id,
            'date': post['date'],
            'text': post.get('text', '')[:500],
            'likes_count': post.get('likes', {}).get('count', 0),
            'comments_count': post.get('comments', {}).get('count', 0),
            'reposts_count': post.get('reposts', {}).get('count', 0),
            'views_count': post.get('views', {}).get('count', 0) if 'views' in post else 0
        }
        
        if 'copy_history' in post and post['copy_history']:
            original_post = post['copy_history'][0]
            post_data['is_repost'] = True
            post_data['reposted_from'] = {
                'owner_id': original_post['owner_id'],
                'post_id': original_post['id'],
                'text': original_post.get('text', '')[:500]
            }
        else:
            post_data['is_repost'] = False
            post_data['reposted_from'] = None
        
        if isinstance(owner_id, int):
            print(f"  Собираем лайки...")
            post_data['likes'] = self.get_likes(owner_id, post_id)
            time.sleep(0.34)
            
            print(f"  Собираем комментарии...")
            post_data['comments'] = self.get_comments(owner_id, post_id)
            time.sleep(0.34)
            
            print(f"  Собираем репосты...")
            post_data['reposts'] = self.get_reposts(owner_id, post_id)
            time.sleep(0.34)
        else:
            print(f"  Пропускаем сбор лайков/комментариев - неверный owner_id")
            post_data['likes'] = []
            post_data['comments'] = []
            post_data['reposts'] = []
        
        return post_data
    
    def parse_user_wall(self, user_input, max_posts=50):
        print(f"\n=== Начинаем парсинг стены пользователя {user_input} ===")
        
        posts, numeric_owner_id = self.get_wall_posts(user_input, max_posts)
        processed_posts = []
        
        for i, post in enumerate(posts):
            print(f"Пост {i+1}/{len(posts)}")
            processed_post = self.process_post(post, numeric_owner_id)
            processed_posts.append(processed_post)
            
            if i < len(posts) - 1:
                time.sleep(1)
        
        return processed_posts, numeric_owner_id
    
    def collect_all_user_ids(self, posts_data):
        user_ids = set()
        
        for post in posts_data:
            user_ids.update(post['likes'])
            
            for comment in post['comments']:
                user_ids.add(comment['from_id'])
            
            for repost in post['reposts']:
                user_ids.add(repost['from_id'])
        
        return list(user_ids)
    
    def analyze_two_users(self, user1_input, user2_input, max_posts_per_user=20):
        print(f"Запускаем анализ пользователей {user1_input} и {user2_input}")
        
        user1_posts, user1_id = self.parse_user_wall(user1_input, max_posts_per_user)
        user2_posts, user2_id = self.parse_user_wall(user2_input, max_posts_per_user)
        
        all_user_ids = set()
        all_user_ids.update(self.collect_all_user_ids(user1_posts))
        all_user_ids.update(self.collect_all_user_ids(user2_posts))
        
        print(f"Собираем информацию о {len(all_user_ids)} пользователях...")
        user_info = self.get_user_info(list(all_user_ids))
        
        result = {
            'analysis_info': {
                'timestamp': datetime.now().isoformat(),
                'user1_input': user1_input,
                'user2_input': user2_input,
                'user1_id': user1_id,
                'user2_id': user2_id,
                'total_posts_analyzed': len(user1_posts) + len(user2_posts),
                'total_users_found': len(all_user_ids)
            },
            'user1_posts': user1_posts,
            'user2_posts': user2_posts,
            'user_info': user_info
        }
        
        return result
    
    def save_to_json(self, data, filename=None):
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            user1 = str(data['analysis_info']['user1_input']).replace('/', '_')
            user2 = str(data['analysis_info']['user2_input']).replace('/', '_')
            filename = f"vk_analysis_{user1}_{user2}_{timestamp}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"Данные сохранены в файл: {filename}")
        return filename
    
    def generate_interaction_csv(self, user_input, output_filename=None, max_posts=100):
        print(f"Генерируем CSV со статистикой взаимодействий для {user_input}...")
        

        posts, owner_id = self.parse_user_wall(user_input, max_posts)
        
        if not posts:
            print("Не удалось получить посты пользователя")
            return None
        
        interactions = defaultdict(lambda: {'likes': 0, 'comments': 0, 'reposts': 0})
        
        for post in posts:
            self._process_post_for_interactions(post, interactions, owner_id)
        
        if not output_filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            user_identifier = str(user_input).replace('/', '_')
            output_filename = f"vk_interactions_{user_identifier}_{timestamp}.csv"
        
        return self._write_interactions_to_csv(owner_id, interactions, output_filename)
    
    def _process_post_for_interactions(self, post, interactions, owner_id):
        post_id = post['post_id']
        
        for like_user_id in post.get('likes', []):
            interactions[like_user_id]['likes'] += 1
        
        for comment in post.get('comments', []):
            commenter_id = comment['from_id']
            interactions[commenter_id]['comments'] += 1
        
        for repost in post.get('reposts', []):
            reposter_id = repost['from_id']
            interactions[reposter_id]['reposts'] += 1
    
    def _write_interactions_to_csv(self, owner_id, interactions, output_filename):
        target_file = 'edges_inference.csv'

        try:
            edges = pd.read_csv(target_file)
        except FileNotFoundError:
            edges = pd.DataFrame(columns=[
                'target_user_id',
                'interactor_id',
                'likes_count',
                'comments_count',
                'reposts_count'
            ])


        new_rows = []
        for interactor_id, stats in interactions.items():
            new_rows.append({
                'target_user_id': owner_id,
                'interactor_id': interactor_id,
                'likes_count': stats['likes'],
                'comments_count': stats['comments'],
                'reposts_count': stats['reposts']
            })

        if new_rows:
            new_df = pd.DataFrame(new_rows)
            edges = pd.concat([edges, new_df], ignore_index=True)

        print(edges)
        edges.to_csv(target_file, index=False)

        print(f"CSV файл обновлён: {target_file}")
        print(f"Добавлено взаимодействий: {len(new_rows)}")
        return target_file

def edges_parser(wall_id):
    # Настройки
    VK_ACCESS_TOKEN = '6fdea0906fdea0906fdea0900e6ce5f2d866fde6fdea0900738eb840c756af0caf0aace'
    # Создаем парсер
    parser = VKParser(VK_ACCESS_TOKEN)

    csv_file = parser.generate_interaction_csv(wall_id, max_posts=10)

In [6]:
for i in profile_names:
    edges_parser(i)

Генерируем CSV со статистикой взаимодействий для 476459668...

=== Начинаем парсинг стены пользователя 476459668 ===
Получаем посты со стены пользователя 476459668...
Получено 6 постов
Пост 1/6
Обрабатываем пост 11...
  Собираем лайки...
  Собираем комментарии...
  Собираем репосты...
Пост 2/6
Обрабатываем пост 10...
  Собираем лайки...
  Собираем комментарии...
  Собираем репосты...
Пост 3/6
Обрабатываем пост 9...
  Собираем лайки...
  Собираем комментарии...
  Собираем репосты...
Пост 4/6
Обрабатываем пост 8...
  Собираем лайки...
  Собираем комментарии...
  Собираем репосты...
Пост 5/6
Обрабатываем пост 6...
  Собираем лайки...
  Собираем комментарии...
  Собираем репосты...
Пост 6/6
Обрабатываем пост 5...
  Собираем лайки...
  Собираем комментарии...
  Собираем репосты...
   target_user_id interactor_id likes_count comments_count reposts_count
0       476459668     265329539           2              1             0
1       476459668     432213567           1              0         

#### Загрузка json и csv файлов в дата фреймы признаков вершин и рёбер

In [7]:
with open('nodes_inference.json', 'r', encoding='utf-8') as f:
    json_file = json.load(f)

    nodes_inference = pd.json_normalize(json_file['people'])

edges_inference = pd.read_csv('edges_inference.csv')
# sampled_edges_inference = edges_inference.drop_duplicates(subset='target_user_id')
# set_ids2 = list(set(sampled_edges_inference['target_user_id']) | set(sampled_edges_inference['interactor_id']))

#### Определение наличия дружбы между людьми и добавление в дата фрейм рёбер

In [8]:
friendship = pd.DataFrame(json_file['edges']).drop(columns=[2]).rename(columns={0: 'user_id', 1: 'friend_id'})

friendship_set = set(friendship.apply(lambda row: frozenset([row['user_id'], row['friend_id']]), axis=1))


def is_friend_pair(row):
    pair = frozenset([row['target_user_id'], row['interactor_id']])
    return pair in friendship_set


edges_inference['is_friend'] = edges_inference.apply(is_friend_pair, axis=1)

#### Агрегация информации у целевых пользователей и обработка в случае отсутствия связи

In [9]:
friends_dict = defaultdict(set)
for _, row in friendship.iterrows():
    friends_dict[row['user_id']].add(row['friend_id'])
    friends_dict[row['friend_id']].add(row['user_id'])


def count_common_friends(u1, u2):
    return len(friends_dict[u1] & friends_dict[u2])


def check_is_friend(u1, u2):
    return int(u2 in friends_dict[u1])


edges_inference['common_friends_count'] = edges_inference.apply(
    lambda row: count_common_friends(row['target_user_id'], row['interactor_id']),
    axis=1
)


mask = (
    ((edges_inference['target_user_id'] == profile_names[0]) & (edges_inference['interactor_id'] == profile_names[1])) |
    ((edges_inference['target_user_id'] == profile_names[1]) & (edges_inference['interactor_id'] == profile_names[0]))
)

pair_edges = edges_inference[mask]


common_friends = count_common_friends(profile_names[0], profile_names[1])
is_friend = check_is_friend(profile_names[0], profile_names[1])


if pair_edges.empty:
    agg_row = {
        'target_user_id': profile_names[0],
        'interactor_id': profile_names[1],
        'likes_count': 0,
        'comments_count': 0,
        'reposts_count': 0,
        'common_friends_count': common_friends,
        'is_friend': is_friend
    }
else:
    agg_row = {
        'target_user_id': profile_names[0],
        'interactor_id': profile_names[1],
        'likes_count': pair_edges['likes_count'].sum(),
        'comments_count': pair_edges['comments_count'].sum(),
        'reposts_count': pair_edges['reposts_count'].sum(),
        'common_friends_count': common_friends,
        'is_friend': is_friend
    }

result_df = pd.DataFrame([agg_row])
print(result_df)

   target_user_id  interactor_id  likes_count  comments_count  reposts_count  \
0       476459668      265329539            2               1              0   

   common_friends_count  is_friend  
0                     7          1  


#### Загрузка предобученной модели и получение предсказаний

В данном случае демонстрируется классификация дружбы двух пользователей, id которых заданы в начале. По сумме их совместных лайков, комментариев и общих друзей происходит бинарная классификация. В начале выводится confusion_matrix, потом её интерпретация, а затем вероятность предсказания.

In [10]:
with open('random_forest_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

feature_columns = ['likes_count', 'comments_count', 'common_friends_count']



X = result_df[feature_columns].fillna(0)
y = result_df['is_friend'].astype(int)


y_pred = loaded_model.predict(X)
y_pred_proba = loaded_model.predict_proba(X)[:, 1]

In [21]:
print("\nClassification Report:")
print(confusion_matrix(y, y_pred))
def prediction_result(y_true, y_pred):
    if y_true[0] == y_pred[0]:
        if y_true[0] == 1:
            print("Верное предсказание: они являются друзьями.")
        else:
            print("Верное предсказание: они не являются друзьями.")
    else:
        if y_true[0] == 1:
            print("Ошибка: предсказано — не друзья, на самом деле — друзья.")
        else:
            print("Ошибка: предсказано — друзья, на самом деле — не друзья.")

prediction_result(y, y_pred)

print(y_pred_proba)


Classification Report:
[[0 0]
 [1 0]]
Ошибка: предсказано — не друзья, на самом деле — друзья.
[0.23972377]


#### Демонстрация классификации на расширенной тестовой выборке

Импортируется датафрейм сбалансированных тестовых данных на 1403 записи для демонстрации процесса классификации и метрик.

In [54]:
model_data = pd.read_csv('model_data.csv')
model_data

,Unnamed: 0,likes_count,comments_count,common_friends_count,is_friend
0,467,2,0,3,0
1,4568,4,0,1,0
2,5468,3,0,1,0
3,4409,2,0,0,1
4,6791,2,1,0,1
...,...,...,...,...,...
1398,3770,0,1,3,0
1399,6251,4,1,0,1
1400,2374,5,1,0,1
1401,6719,1,0,1,0


In [52]:
feature_columns = ['likes_count', 'comments_count', 'common_friends_count']

X_test = model_data[feature_columns].fillna(0)
y_test = model_data['is_friend'].astype(int)

y_pred = loaded_model.predict(X_test)
y_pred_proba = loaded_model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.71      0.81       871
           1       0.66      0.93      0.77       532

    accuracy                           0.79      1403
   macro avg       0.80      0.82      0.79      1403
weighted avg       0.84      0.79      0.80      1403


Confusion Matrix:
[[615 256]
 [ 35 497]]
